# Oxylabs

>[Oxylabs](https://oxylabs.io/) 是市场领先的网络情报收集平台，以最高的商业、道德和合规标准为驱动，赋能全球企业解锁数据驱动的洞察。

## 概览

此包包含 LangChain 与 Oxylabs 的集成，提供了使用 LangChain 框架通过 Oxylabs Web Scraper API 抓取 Google 搜索结果的工具。

此包提供以下类：
- `OxylabsSearchRun` - 一个返回格式化文本的 Google 搜索结果的工具
- `OxylabsSearchResults` - 一个返回 JSON 格式的 Google 搜索结果的工具
- `OxylabsSearchAPIWrapper` - 用于初始化 Oxylabs API 的 API 封装器

|             定价             |
|:-------------------------------:|
| ✅ 免费 5,000 次结果，有效期 1 周 |

## 设置

安装所需的依赖项。

In [ ]:
%pip install -qU langchain-oxylabs

### 凭证

设置正确的 API 密钥和环境变量。创建您的 API 用户凭据：在 [Oxylabs dashboard](https://dashboard.oxylabs.io/en/registration) 注册免费试用或购买产品，以创建您的 API 用户凭据（OXYLABS_USERNAME 和 OXYLABS_PASSWORD）。

In [ ]:
import getpass
import os

os.environ["OXYLABS_USERNAME"] = getpass.getpass("Enter your Oxylabs username: ")
os.environ["OXYLABS_PASSWORD"] = getpass.getpass("Enter your Oxylabs password: ")

## 实例化

In [ ]:
from langchain_oxylabs import OxylabsSearchAPIWrapper, OxylabsSearchRun

oxylabs_wrapper = OxylabsSearchAPIWrapper()
tool_ = OxylabsSearchRun(wrapper=oxylabs_wrapper)

## 调用

### 使用参数直接调用

`OxylabsSearchRun` 工具接受一个名为 "query" 的参数，该参数应该是一个自然语言查询，并以组合字符串格式返回结果：

In [ ]:
tool_.invoke({"query": "Restaurants in Paris."})

### 使用 ToolCall 调用

In [ ]:
tool_ = OxylabsSearchRun(
    wrapper=oxylabs_wrapper,
    kwargs={
        "result_categories": [
            "local_information",
            "combined_search_result",
        ]
    },
)

In [ ]:
from pprint import pprint

model_generated_tool_call = {
    "args": {
        "query": "Visit restaurants in Vilnius.",
        "geo_location": "Vilnius,Lithuania",
    },
    "id": "1",
    "name": "oxylabs_search",
    "type": "tool_call",
}
tool_call_result = tool_.invoke(model_generated_tool_call)

# The content is a JSON string of results
pprint(tool_call_result.content)

## 在 Agent 中使用
安装所需的依赖。

In [ ]:
%pip install -qU "langchain[openai]" langgraph

In [ ]:
import getpass
import os

from langchain.chat_models import init_chat_model

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
from langgraph.prebuilt import create_react_agent

# Initialize OxylabsSearchRun tool
tool_ = OxylabsSearchRun(wrapper=oxylabs_wrapper)

agent = create_react_agent(llm, [tool_])

user_input = "What happened in the latest Burning Man floods?"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## JSON 结果
`OxylabsSearchResults` 工具可以作为 `OxylabsSearchRun` 的替代品，以 JSON 格式检索结果：

In [ ]:
import json

from langchain_oxylabs import OxylabsSearchResults

tool_ = OxylabsSearchResults(wrapper=oxylabs_wrapper)

response_results = tool_.invoke({"query": "What are the most famous artists?"})
response_results = json.loads(response_results)

for result in response_results:
    for key, value in result.items():
        print(f"{key}: {value}")

## API 参考
有关此集成包的更多信息，请访问：https://github.com/oxylabs/langchain-oxylabs

Oxylabs Web Scraper API 文档：https://developers.oxylabs.io/scraper-apis/web-scraper-api